# LSTM model
### Predicts MIMIC-III ICU patient mortality given the first 24 hours

In [1]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Activation, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, auc, roc_curve, brier_score_loss, precision_recall_curve

import numpy as np
import pandas as pd
from collections import defaultdict

from random import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)

ModuleNotFoundError: No module named 'keras'

In [19]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')

import dbconfig as cfg
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://{}:{}@{}:3306/mimic'.format(cfg.mysql['user'], cfg.mysql['password'],
                                                                cfg.mysql['host']), echo=False)

### Load data

In [20]:
mimic_df = pd.read_pickle('/home/andrea/data/mimic_timeseries_normalized')
mimic_df = pd.get_dummies(mimic_df, columns=['ADMISSION_LOCATION', 'ADMISSION_TYPE'])
label_col = mimic_df['HOSPITAL_EXPIRE_FLAG']
del mimic_df['HOSPITAL_EXPIRE_FLAG']
mimic_df['HOSPITAL_EXPIRE_FLAG'] = label_col
# mimic_df.head()

In [21]:
mimic_df.columns.values

array(['GENDER', 'AGE', 'lab_hemoglobin', 'lab_monocyte', 'lactate_dh',
       'lab_eosinophil', 'lab_glucose', 'lab_ck', 'lab_basophils',
       'troponin_t', 'sodium_whole_blood', 'art_dia', 'resp_pattern',
       'bp_dia', 'chart_temp', 'glasgow_score', 'art_mean', 'bp_sys',
       'art_sys', 'riker_sas', 'cvp', 'eye_open',
       'ADMISSION_LOCATION_** INFO NOT AVAILABLE **',
       'ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE',
       'ADMISSION_LOCATION_EMERGENCY ROOM ADMIT',
       'ADMISSION_LOCATION_HMO REFERRAL/SICK',
       'ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI',
       'ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM',
       'ADMISSION_LOCATION_TRANSFER FROM OTHER HEALT',
       'ADMISSION_LOCATION_TRANSFER FROM SKILLED NUR',
       'ADMISSION_LOCATION_TRSF WITHIN THIS FACILITY',
       'ADMISSION_TYPE_ELECTIVE', 'ADMISSION_TYPE_EMERGENCY',
       'ADMISSION_TYPE_URGENT', 'HOSPITAL_EXPIRE_FLAG'], dtype=object)

In [22]:
# binarize gender
d = {'GENDER': {'M': 0, 'F': 1}}
mimic_df.replace(d, inplace=True)
mimic_df.head()

,GENDER,AGE,lab_hemoglobin,lab_monocyte,lactate_dh,lab_eosinophil,lab_glucose,lab_ck,lab_basophils,troponin_t,...,ADMISSION_LOCATION_HMO REFERRAL/SICK,ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,ADMISSION_LOCATION_TRANSFER FROM OTHER HEALT,ADMISSION_LOCATION_TRANSFER FROM SKILLED NUR,ADMISSION_LOCATION_TRSF WITHIN THIS FACILITY,ADMISSION_TYPE_ELECTIVE,ADMISSION_TYPE_EMERGENCY,ADMISSION_TYPE_URGENT,HOSPITAL_EXPIRE_FLAG
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,0,76,"[0, 0, 0.0769230769231, 0, 0, 0, 0.07692307692...","[0, 0, 0, 0, 0, 0, 0, 0.0641025641026, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0.0322580645161, 0, 0...","[0.000296194093365, 0, 0, 0, 0, 0, 0, 0.001379...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,1,0,0
4,1,47,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.025641025641, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0.00406604917852, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,1,0,0
6,1,65,"[0.0615384615385, 0, 0, 0.0615384615385, 0.069...","[0, 0, 0, 0, 0, 0, 0, 0, 0.0128205128205, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,1,0,0,0,0,1,0,0,0
9,0,41,"[0, 0, 0, 0, 0.107692307692, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,1,0,1
11,1,50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,1,0,0


### Shuffle and split data into train/test datasets

In [23]:
def shuffle_split(l, train=0.8, test=0.1, val=0.1):
    shuffle(l)
    if val == 0:
        train = 0.7
    train_index = int(len(l)*train)
    test_index = train_index + int(len(l)*test)
    if val == 0:
        return l[:train_index], l[train_index:]
    return l[:train_index], l[train_index:test_index], l[test_index:]

In [24]:
# expert defined ranges (low, high)
range_dict = {
    'temp': (97.8, 99.0),
    'temp_aux': (97.4, 98.7), #auxillary/armpit, (F)
    'temp_rectal': (98.3, 100.0), 
    'bp_sys': (120.0,180.0),   #mmHg
    'bp_dia': (80.0, 120.0),   #mmHg
    'cvp': (3.0, 8.0),         #
    'resp': (16.0, 20.0),      #rate
    'art_sys': (20.0, 119.0),  #mmHg
    'art_dia': (60.0, 79.0),   #mmHg
     
    'lab_ck_male': (52.0, 336.0),  #U/L
    'lab_ck_female': (38.0,176.0), #U/L
    'lab_glucose': (70.0, 100.0),  #mg/dL
    'lab_monocyte': (2.0, 10.0),   #percent
    'lab_eosinophil': (1.0, 6.0),  #percent
    'lab_basophils': (0.01, 2.0),#percent
#     'lab_neutrohils': (40.0,80.0), #percent
    'lab_lymphocytes': (20.0,40.0),#percent
    'lab_hemoglobin_female': (12.0, 17.5),  #g/dL
    'lab_hemoglobin_male': (13.5, 17.5),   #g/dL
    'lactate_dh': (140.0, 280.0), #U/L
    'sodium_whole_blood': (135.0, 145.0), #mEq/L
    'troponin_t': (0, 0.01) #ng/mL
}

In [26]:
def zip_timeseries(df):
    l = []
    for patient in df.index.values:
        z = []
        temp = zip(#df.get_value(patient, 'lab_temp'), 
                    df.get_value(patient, 'lab_ck'), 
                    df.get_value(patient, 'lab_monocyte'), 
                    df.get_value(patient, 'lab_glucose'), 
                    df.get_value(patient, 'lab_eosinophil'),
                    df.get_value(patient, 'lab_basophils'),
                    #df.get_value(patient, 'lab_lymphocytes'),
                    df.get_value(patient, 'lab_hemoglobin'),
                    df.get_value(patient, 'lactate_dh'),
                    df.get_value(patient, 'sodium_whole_blood'),
                    df.get_value(patient, 'troponin_t'),
                    df.get_value(patient, 'bp_sys'), 
                    df.get_value(patient, 'bp_dia'), 
                    df.get_value(patient, 'cvp'), 
                    df.get_value(patient, 'art_mean'),
                    df.get_value(patient, 'art_sys'),
                    df.get_value(patient, 'art_dia'),
#                     df.get_value(patient, 'riker_sas'),
#                     df.get_value(patient, 'resp_pattern'),
#                     df.get_value(patient, 'eye_open'),
#                     df.get_value(patient, 'glasgow_score'),
#                     df.get_value(patient, 'abp_low'),
#                     df.get_value(patient, 'abp_high'),
                    df.get_value(patient, 'chart_temp')
#                     [df.get_value(patient, 'GENDER')]*24, 
#                     [df.get_value(patient, 'AGE')]*24
                  )
        for timeseries in temp:
            z.append(timeseries)
        l.append(z)
    return l

In [27]:
patient_list = mimic_df.index.values
train, test, val = shuffle_split(patient_list)

train_df = mimic_df.ix[train]
test_df = mimic_df.ix[test]
val_df = mimic_df.ix[val]

X_train = np.array(zip_timeseries(train_df))
X_test = np.array(zip_timeseries(test_df))
X_val = np.array(zip_timeseries(val_df))

features = train_df.columns[:-1]

y_train = train_df['HOSPITAL_EXPIRE_FLAG'].values
y_test = test_df['HOSPITAL_EXPIRE_FLAG'].values
y_val = val_df['HOSPITAL_EXPIRE_FLAG'].values

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)
y_val = y_val.reshape(len(y_val), 1)

# total data/labels set for cross-validation
X = np.array(zip_timeseries(mimic_df[features]))
y = mimic_df['HOSPITAL_EXPIRE_FLAG'].values

print('{} observations in the training data'.format(len(train_df.index.values)))
print('{} observations in the test data'.format(len(test_df.index.values)))
print('{} observations in the validation data'.format(len(val_df.index.values)))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


30838 observations in the training data
3854 observations in the test data
3856 observations in the validation data


### Manual k-fold Cross-Validation of LSTM Model

In [ ]:
nb_epoch = 10
batch_size = 32
wait_until_stop = 5 # number of epochs with no improvement to early stop
stop_monitor = 'loss'
k = 10

In [ ]:
def round_predictions(preds):
    rounded_predictions = []
    for pred in preds:
        if pred < 0.5:
            rounded_predictions.append(0)
        elif pred >= 0.5:
            rounded_predictions.append(1)
    return rounded_predictions

In [ ]:
kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=7)
acc_scores = []
auc_vals = []
# precision_vals = []
# recall_vals = []
# f1_vals = []
pred_vals = []
true_vals = []
run_ids = []

fpr_vals = []
tpr_vals = []
thresholds = []

dropout = 0.0

count = 1
for train, test in kfold.split(X, y):
    print('Fold {} out of {}'.format(count, k))
    
    model = Sequential()
    model.add(LSTM(128, dropout_W=dropout, dropout_U=dropout, 
                   input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(LSTM(64, dropout_W=dropout, dropout_U=dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'], batch_size=batch_size)
    early_stop = EarlyStopping(monitor=stop_monitor, patience=wait_until_stop, mode='auto')
    model.fit(X[train], y[train], nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, callbacks=[early_stop])
    
    # evaluate
    preds = model.predict(X[test])
    pred_vals.append(preds)
    
    y_true = pd.Series(np.squeeze(y[test]))
    true_vals.append(y_true)
    
#     y_pred = pd.Series(round_predictions(preds))
#     precision_vals.append(precision_score(y_true=y_true, y_pred=y_pred))
#     recall_vals.append(recall_score(y_true=y_true, y_pred=y_pred))
#     f1_vals.append(f1_score(y_true=y_true, y_pred=y_pred))
    
    fpr, tpr, threshold = roc_curve(y[test], preds)
    fpr_vals.append(fpr)
    tpr_vals.append(tpr)
    thresholds.append(threshold)
    
    auc_vals.append(auc(fpr, tpr))
    
    scores = model.evaluate(X[test], y[test], verbose=0)
    print('Accuracy: {:.3f}'.format(scores[1]*100))
    acc_scores.append(scores[1]*100)
    run_id = 'no_dropout_high_param, fold={}'.format(count)
    run_ids.append([run_id]*len(preds))
    count += 1
print('{:.3f} +/- {:.3f}'.format(np.mean(acc_scores), np.std(acc_scores)))

In [ ]:
def flatten(l, f=True):
    if f:
        return [float(item) for sublist in l for item in sublist]
    else:
        return [item for sublist in l for item in sublist]

In [ ]:
results_df = pd.DataFrame()
results_df['pred_val'] = flatten(pred_vals)
results_df['true_label'] = flatten(true_vals)
results_df['run_ids'] = flatten(run_ids, f=False)
results_df.head()

In [29]:
model = Sequential()
model.add(LSTM(32, dropout_W=0.4, dropout_U=0.4, 
               input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
# model.add(LSTM(16, dropout_W=0.4, dropout_U=0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_2 (LSTM)                    (None, 32)            6272        lstm_input_2[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             33          lstm_2[0][0]                     
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [ ]:
# early_stop = EarlyStopping(monitor=stop_monitor, patience=wait_until_stop, mode='auto')
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), nb_epoch=nb_epoch, callbacks=[early_stop])

# # plot progress
# f, (ax1, ax2) = plt.subplots(1, 2, sharey=False, sharex=True, figsize=(16, 6))
# ax1.plot(history.history['loss'], label='train')
# ax1.plot(history.history['val_loss'], label='val')
# ax1.set_title('Train/val loss per epoch')
# ax1.legend()

# ax2.plot(history.history['acc'], label='train')
# ax2.plot(history.history['val_acc'], label='val')
# ax2.set_title('Train/val accuracy per epoch')
# ax2.legend()
# plt.show()

### Evaluate LSTM model

In [15]:
def evaluate(table):
    results_df = pd.read_sql_table(table, con=engine)
    n_folds = set(results_df.run_ids.values)

    auc_vals = []
    brier_scores = []
    aucprc_vals = []

    for fold in n_folds:
        temp_df = results_df.loc[results_df['run_ids']==fold]
        pred_vals = temp_df.pred_val.values
        true_vals = temp_df.true_label.values
        fpr, tpr, threshold = roc_curve(true_vals, pred_vals)
        auc_vals.append(auc(fpr,tpr))
        brier_scores.append(brier_score_loss(true_vals,pred_vals))
        aucprc_vals.append(precision_recall_curve(true_vals,pred_vals))

    return np.mean(auc_vals), np.mean(brier_scores), np.mean(aucprc_vals)

In [17]:
tables = ['lstm_dropout_high_param_results', 'lstm_dropout_results', 'lstm_no_dropout_high_param_results', 'lstm_no_dropout_results']
for table in tables:
    print(table)
    a,b,c = evaluate(table)
    print('avg AUCROC: {:.4f}, avg Brier score: {:.4f}, avg AUCPRC\n'.format(a,b,c))

lstm_dropout_high_param_results


ValueError: operands could not be broadcast together with shapes (3385,) (3384,) 

In [ ]:
# best model is lstm_dropout_results


In [ ]:
# score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
# # print('{}: {}, {}: {}').format(model.metrics_names[0], score[0], model.metrics_names[1], score[1])
# print('Accuracy: {:.3f}%'.format(score[1]*100))

In [ ]:
# print('Precision: {:0.4f}'.format(precision_score(y_true=y_true, y_pred=y_pred)))
# print('Recall: {:0.4f}'.format(recall_score(y_true=y_true, y_pred=y_pred)))
# print('F1 score: {:0.4f}'.format(f1_score(y_true=y_true, y_pred=y_pred)))

In [ ]:
# # best/worst auc
# worst_index = auc_vals.index(min(auc_vals))
# best_index = auc_vals.index(max(auc_vals))

# plt.figure(figsize=(8,7))
# plt.title('ROC with Dropout')
# plt.plot(fpr_vals[worst_index], tpr_vals[worst_index], 'b', label = 'AUC = %0.2f' % auc_vals[worst_index])
# plt.plot(fpr_vals[best_index], tpr_vals[best_index], 'r', label = 'AUC = %0.2f' % auc_vals[best_index])
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'k--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

### Write results to RDS 

In [ ]:
results_df.to_sql(name='lstm_no_dropout_high_param_results', con=engine, index=False, if_exists='append')